In [1]:
import json
from mlp_mixer import * 
import torch
from get_dataloaders import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'

path = "./models/20220401-122435/"
with open(path+'out_hyperparams.json') as json_file:
    params = json.load(json_file)

In [2]:
model = MLP_mixer(img_h_w=params['image_width_and_height'], patch_dim=params['patch_width_and_height'], n_channels=params['hidden_dim_size (n_channels)'], num_mixers_layers=params['number_of_layers'],
    hidden_dim_mlp_token=params['mlp_ds_dimension'], hidden_dim_mlp_channel=params['mlp_dc_dimension'], n_classes=1000)

model.load_state_dict(torch.load(path+"checkpoint_epch_300.pth", map_location='cpu'))
model.eval()

MLP_mixer(
  (patch_embedder): Conv2d(3, 128, kernel_size=(8, 8), stride=(8, 8))
  (mixerlayers): ModuleList(
    (0): MixerLayer(
      (layer_norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (layer_norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (MLP1): MLP(
        (linear_gelu_stack): Sequential(
          (0): Linear(in_features=64, out_features=64, bias=True)
          (1): GELU()
          (2): Linear(in_features=64, out_features=64, bias=True)
        )
      )
      (MLP2): MLP(
        (linear_gelu_stack): Sequential(
          (0): Linear(in_features=128, out_features=512, bias=True)
          (1): GELU()
          (2): Linear(in_features=512, out_features=128, bias=True)
        )
      )
    )
    (1): MixerLayer(
      (layer_norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (layer_norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (MLP1): MLP(
        (linear_gelu_stack): Sequential(
          (0):

In [3]:
params['batch_size'] = 512
train_loader, val_loader, num_classes = getUpsampledCIFAR100Loaders(params)
num_in_features = model.fc_head.in_features

#cifar has 100 classes
model.fc_head = nn.Linear(num_in_features, num_classes)
model.to(device)

Files already downloaded and verified
BATCH SIZE: 512
Tran subset len: 50000
Tran loader len: 98
Test: 97.65625
Val/test subset len: 10000
Val/test subset len: 20
Val/Test: 19.53125
Test subset len: 10000
Test subset len: 20
Test: 19.53125


MLP_mixer(
  (patch_embedder): Conv2d(3, 128, kernel_size=(8, 8), stride=(8, 8))
  (mixerlayers): ModuleList(
    (0): MixerLayer(
      (layer_norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (layer_norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (MLP1): MLP(
        (linear_gelu_stack): Sequential(
          (0): Linear(in_features=64, out_features=64, bias=True)
          (1): GELU()
          (2): Linear(in_features=64, out_features=64, bias=True)
        )
      )
      (MLP2): MLP(
        (linear_gelu_stack): Sequential(
          (0): Linear(in_features=128, out_features=512, bias=True)
          (1): GELU()
          (2): Linear(in_features=512, out_features=128, bias=True)
        )
      )
    )
    (1): MixerLayer(
      (layer_norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (layer_norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (MLP1): MLP(
        (linear_gelu_stack): Sequential(
          (0):

In [4]:
#define hyperparameters for the fine tuning
new_params = {'learning_rate': 0.001, 'weight_decay': 1e-5}

In [5]:
from tqdm.notebook import tqdm
import time
from comet_ml import Experiment

experiment = Experiment(
    api_key="xX6qWBFbiOreu0W3IrO14b9nB",
    project_name="mlp-mixer-finetune",
    workspace="wedrid",
)

log = True
num_epochs = 500
optimizer = torch.optim.Adam(model.parameters(), lr = new_params['learning_rate'], weight_decay=new_params['weight_decay'])

loss_func = nn.CrossEntropyLoss()
model.to(device)
# training loop
for epoch in tqdm(range(num_epochs)):
    start = time.time()
    model.train()
    train_accuracy = 0
    for i, (images, labels) in enumerate(tqdm(train_loader)):
        # [100, 3, 36, 36] is what is returned by iterator
        images = images.to(device)
        labels = labels.to(device)
        
        # forward pass
        predicted = model(images)
        loss = loss_func(predicted, labels)
        train_accuracy += ((predicted.argmax(dim=-1) == labels).float().mean()).item()

        # backwards pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if False and (i+1) % 100:
            print(f'epoch: {epoch+1} of {num_epochs}, step {i+1} of {steps_total}, loss = {loss.item():.4f}')
    print(f"Loss of epoch {epoch+1}: {loss.item():.4f}")
    train_accuracy /= len(train_loader)
    #print(f"TRAIN LOADER LENGTH: {len(train_loader)}")
    end = time.time()
    elapsed = end - start
    if log: 
        experiment.log_metric("train epoch loss", loss.item(), step=epoch)
        experiment.log_metric("mean train epoch accuracy", train_accuracy, step=epoch)
        experiment.log_metric("epoch time", elapsed, step = epoch)

    # validation
    with torch.no_grad():
        model.eval()
        val_accuracy = 0
        temp = 0
        for i, (images, labels) in enumerate(tqdm(val_loader)): #numero esempi/batchsize TODO check
            # [100, 3, 36, 36] is what is returned by iterator
            images = images.to(device)
            labels = labels.to(device)
            
            # forward pass
            predicted = model(images)
            loss = loss_func(predicted, labels)
            val_accuracy += ((predicted.argmax(dim=-1) == labels).float().mean()).item()
        #print(f"Lenght val loader: {len(val_loader)}, counter: {temp}")
        val_accuracy /= len(val_loader) 
        if log: 
            experiment.log_metric("val epoch loss", loss.item(), step=epoch)
            experiment.log_metric("mean val epoch accuracy", val_accuracy, step=epoch)
    
    if epoch % 10 == 0 and False:
        torch.save(model.state_dict(), model_path + f"checkpoint_epch_{epoch}.pth")

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wedrid/mlp-mixer-finetune/dd885817d639418cadc261120939bcab



  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/98 [00:00<?, ?it/s]